## #1 Predicts probabilities of each label for a batch of images
See output and header info below

## #2 Converts prediction file with probabilities to predicted true labels (Kaggle iMaterialist format)

In [1]:
import csv
import os
import sys
import time
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from PIL import Image, ImageOps
import os, sys
print("TensorFlow version:", tf.__version__)

TensorFlow version: 1.7.0


In [3]:
#1 Predict probabilities for labels, save .csv

#Runtime 109s for 100 (155kB, with .4 float precision, on Intel i7), 1445s for 1000 imgs (1.6MB)

#params
input_path = 'validation256/'
first = 1
last = 1000
image_numbers = range(first, last+1)
save_path = 'predictions/'
save_as = 'predicted_v1000.csv'

start = time.time()

#modified from https://github.com/BartyzalRadek/Multi-label-Inception-net
# Loads label file, strips off carriage return
label_lines = [line.rstrip() for line 
                   in tf.gfile.GFile("labels.txt")]
top_k = np.asarray([int(lbl) for lbl in label_lines])

all_predictions = np.empty([len(image_numbers),len(label_lines)+1])

# Unpersists graph from file
with tf.gfile.FastGFile("retrained_graph.pb", 'rb') as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())
    _ = tf.import_graph_def(graph_def, name='')


#iterate over images
counter = 0
for i, image_num in enumerate(image_numbers):
    image_path = input_path+str(image_num)+'.jpg'
    #print(image_path)
    # Read in the image_data
    image_data = tf.gfile.FastGFile(image_path, 'rb').read()
    
    with tf.Session() as sess:
        # Feed the image_data as input to the graph and get first prediction
        softmax_tensor = sess.graph.get_tensor_by_name('final_result:0')

        predictions = sess.run(softmax_tensor, \
                 {'DecodeJpeg/contents:0': image_data})

    all_predictions[i] = np.insert(predictions, 0, image_num)
    
    if (counter == 1000):
        print(image_num)
        np.savetxt(save_path+save_as, all_predictions, delimiter=",", fmt='%1.4f')
        counter = 0
        
    counter += 1

np.savetxt(save_path+save_as, all_predictions, delimiter=",", fmt='%1.4f')

#header


end = time.time()
print(end - start)
print('Saved to '+save_path+save_as)

1445.5216188430786


In [42]:
#header:
#image_num, labels separated by commas, sourced from labels.txt (generated in preprocessing)
header=[lbl for lbl in label_lines]
header.insert(0,'image')
with open(save_path+save_as+'.head', 'w') as headfile:
    wr = csv.writer(headfile)
    wr.writerow(header)
print('Saved to '+save_path+save_as+'.head')
print(header)

Saved to predictions/predicted_v1000.csv.head
['image', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '95', '96', '97', '98', '99', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '120', '121', '122', '123', '124', '125', '126', '127', '128', '129', '130', '131', '132', '133', '134', '135', '136', '137', '138', '139', '140', '141', '142', '143', '144', '146', '147', '148', '149', '150', '151', '152', '153', '154',

In [43]:
#example entry for 1 image:
all_predictions[0]

array([ 1.        ,  0.17220522,  0.17524981,  0.17121118,  0.16985664,
        0.17000282,  0.16917139,  0.17647776,  0.16882296,  0.17106469,
        0.16800098,  0.17487083,  0.16953056,  0.17390741,  0.1758121 ,
        0.17792206,  0.29614499,  0.19244042,  0.2207858 ,  0.22344185,
        0.17249553,  0.17033365,  0.16778883,  0.16925192,  0.17555028,
        0.1703099 ,  0.1683688 ,  0.17517875,  0.16967089,  0.16830975,
        0.16876042,  0.17639604,  0.16827677,  0.16674174,  0.1720027 ,
        0.20024487,  0.17230676,  0.17561071,  0.17264313,  0.17023093,
        0.17080215,  0.16964871,  0.20744066,  0.17268752,  0.16641204,
        0.17331682,  0.17044078,  0.20116149,  0.16867325,  0.17390676,
        0.17427619,  0.23013963,  0.16984232,  0.17299178,  0.17015721,
        0.17070986,  0.16621698,  0.1890462 ,  0.16809623,  0.18436661,
        0.2466597 ,  0.17446767,  0.17390631,  0.17478429,  0.69878429,
        0.16992924,  0.16417791,  0.17368294,  0.18907806,  0.16

#iMaterialist sample submission format:
image_id,label_id
1,106 115 126 145 161 176 185 32
2,115 13 14 158 18 184 190 220 227 44 47 56 81 9 92
3,104 112 129 224 31 83
4,113 121 145 150 218 63 77
5,1 126 167 170 196 208 213 216 22 32 48 54 79

In [52]:
#2 Read label probabilities .csv and corresponding headers (.head), select true labels by thresholding, save in competition-required format
#Runtime 0.3s for 1000

#params
threshold = 0.35

input_prob_path = 'predictions/'
input_prob = 'predicted_v1000.csv'
save_path = 'predictions/'
save_as = 'final_'+str(threshold)+'_'+input_prob


start = time.time()

probdf = pd.read_csv(input_prob_path+input_prob, header=None)
probdf.columns=pd.read_csv(input_prob_path+input_prob+'.head', header=None).values.tolist()
probdf['image'] = probdf['image'].astype(int)

with open(save_path+save_as, "w") as final_labels:
    final_labels.write('image_id,label_id'+'\n')
    for ri, row in probdf.iterrows():
        image_labels = str(int(row[0]))+','
        for ci, col in enumerate(row[1:]):
            if col >= threshold:
                image_labels = image_labels+str(int(probdf.columns[ci+1][0]))+' '
        final_labels.write(image_labels[:-1]+'\n')

end = time.time()
print(end - start)
print('Saved to '+save_path+save_as)

0.28173303604125977
Saved to predictions/final_0.35_predicted_v1000.csv
